## 这里我们将对CAMB中的fortran源码进行重构,并全部用python实现

我们关注的是scalar mode下的Cl理论值

### 首先我们取出源码中计算Cl的部分

```fortran
subroutine CalcScalCls(CTrans)
    use Bispectrum
    implicit none
    Type(ClTransferData) :: CTrans
    integer j, q_ix, w_ix, w_ix2
    real(dl) apowers
    real(dl) dlnk, ell, ctnorm, dbletmp, Delta1, Delta2
    real(dl), allocatable :: ks(:), dlnks(:), pows(:)
    real(dl) fac(3 + State%num_redshiftwindows + State%CP%CustomSources%num_custom_sources)
    integer nscal, i

    allocate(ks(CTrans%q%npoints),dlnks(CTrans%q%npoints), pows(CTrans%q%npoints))
    do q_ix = 1, CTrans%q%npoints
        if (State%flat) then
            ks(q_ix) = CTrans%q%points(q_ix)
            dlnks(q_ix) = CTrans%q%dpoints(q_ix)/CTrans%q%points(q_ix)
        else
            ks(q_ix) = sqrt(CTrans%q%points(q_ix)**2 - State%curv)
            dlnks(q_ix) = CTrans%q%dpoints(q_ix)*CTrans%q%points(q_ix)/ks(q_ix)**2
        end if
        pows(q_ix) = CP%InitPower%ScalarPower(ks(q_ix))
        if (global_error_flag/=0) return
    end do

#ifndef __INTEL_COMPILER
    !Can't use OpenMP here on ifort. Does not terminate.
    !$OMP PARALLEL DO DEFAULT(SHARED), SCHEDULE(STATIC,4), &
    !$OMP PRIVATE(ell,q_ix,dlnk,apowers,ctnorm,dbletmp,Delta1,Delta2,w_ix,w_ix2,fac, nscal, i)
#endif
    do j=1,CTrans%ls%nl
        !Integrate dk/k Delta_l_q**2 * Power(k)
        ell = real(CTrans%ls%l(j),dl)
        if (j<= CTrans%max_index_nonlimber) then
            do q_ix = 1, CTrans%q%npoints
                if (.not.(State%closed.and.nint(CTrans%q%points(q_ix)*State%curvature_radius)<=CTrans%ls%l(j))) then
                    !cut off at nu = l + 1
                    dlnk = dlnks(q_ix)
                    apowers = pows(q_ix)

                    iCl_scalar(j,C_Temp:C_E) = iCl_scalar(j,C_Temp:C_E) +  &
                        apowers*CTrans%Delta_p_l_k(1:2,j,q_ix)**2*dlnk
                    iCl_scalar(j,C_Cross) = iCl_scalar(j,C_Cross) + &
                        apowers*CTrans%Delta_p_l_k(1,j,q_ix)*CTrans%Delta_p_l_k(2,j,q_ix)*dlnk

                    if (CTrans%NumSources>2 .and. State%CP%want_cl_2D_array) then

                        do w_ix=1,3 + State%num_redshiftwindows
                            Delta1= CTrans%Delta_p_l_k(w_ix,j,q_ix)
                            if (w_ix>3) then
                                associate (Win => State%Redshift_w(w_ix - 3))
                                    if (Win%kind == window_lensing) &
                                        Delta1 = Delta1 / 2 * ell * (ell + 1)
                                    if (Win%kind == window_counts .and. CP%SourceTerms%counts_lensing) then
                                        !want delta f/f - 2kappa;
                                        ! grad^2 = -l(l+1);
                                        Delta1 = Delta1 + ell * (ell + 1) * &
                                            CTrans%Delta_p_l_k(3 + Win%mag_index + &
                                            State%num_redshiftwindows, j, q_ix)
                                    end if
                                end associate
                            end if
                            do w_ix2=w_ix,3 + State%num_redshiftwindows
                                if (w_ix2>= 3.and. w_ix>=3) then
                                    !Skip if the auto or cross-correlation is included in direct Limber result
                                    !Otherwise we need to include the sources e.g. to get counts-Temperature correct
                                    if (CTrans%limber_l_min(w_ix2)/= 0 .and. j>=CTrans%limber_l_min(w_ix2) &
                                        .and. CTrans%limber_l_min(w_ix)/= 0 .and. j>=CTrans%limber_l_min(w_ix)) cycle

                                end if
                                Delta2 = CTrans%Delta_p_l_k(w_ix2, j, q_ix)
                                if (w_ix2 > 3) then
                                    associate (Win => State%Redshift_w(w_ix2 - 3))
                                        if (Win%kind == window_lensing) &
                                            Delta2 = Delta2 / 2 * ell * (ell + 1)
                                        if (Win%kind == window_counts .and. CP%SourceTerms%counts_lensing) then
                                            !want delta f/f - 2kappa;
                                            ! grad^2 = -l(l+1);
                                            Delta2 = Delta2 + ell * (ell + 1) * &
                                                CTrans%Delta_p_l_k(3 + Win%mag_index + &
                                                State%num_redshiftwindows, j, q_ix)
                                        end if
                                    end associate
                                end if
                                iCl_Array(j,w_ix,w_ix2) = iCl_Array(j,w_ix,w_ix2)+Delta1*Delta2*apowers*dlnk
                            end do
                        end do
                        if (CP%CustomSources%num_custom_sources >0) then
                            do w_ix=1,3 + State%num_redshiftwindows + CP%CustomSources%num_custom_sources
                                if (w_ix > 3 + State%num_redshiftwindows) then
                                    Delta1= CTrans%Delta_p_l_k(w_ix+State%num_extra_redshiftwindows,j,q_ix)
                                else
                                    Delta1= CTrans%Delta_p_l_k(w_ix,j,q_ix)
                                end if
                                do w_ix2=max(w_ix,3 + State%num_redshiftwindows +1), &
                                    3 + State%num_redshiftwindows +CP%CustomSources%num_custom_sources
                                    Delta2=  CTrans%Delta_p_l_k(w_ix2+State%num_extra_redshiftwindows,j,q_ix)
                                    iCl_Array(j,w_ix,w_ix2) = iCl_Array(j,w_ix,w_ix2) &
                                        +Delta1*Delta2*apowers*dlnk
                                end do
                            end do
                        end if
                    end if

                    if (CTrans%NumSources>2 ) then
                        if (CP%SourceTerms%limber_phi_lmin==0 .or.  &
                            CTrans%limber_l_min(3)== 0 .or. j<CTrans%limber_l_min(3)) then
                            iCl_scalar(j,C_Phi) = iCl_scalar(j,C_Phi) +  &
                                apowers*CTrans%Delta_p_l_k(3,j,q_ix)**2*dlnk
                            iCl_scalar(j,C_PhiTemp) = iCl_scalar(j,C_PhiTemp) +  &
                                apowers*CTrans%Delta_p_l_k(3,j,q_ix)*CTrans%Delta_p_l_k(1,j,q_ix)*dlnk
                            iCl_scalar(j,C_PhiE) = iCl_scalar(j,C_PhiE) +  &
                                apowers*CTrans%Delta_p_l_k(3,j,q_ix)*CTrans%Delta_p_l_k(2,j,q_ix)*dlnk
                        end if
                    end if
                end if
            end do

        end if !limber (j<= max_bessels_l_index)

        !Output l(l+1)C_l/OutputDenominator
        ctnorm=(ell*ell-1)*(ell+2)*ell
        dbletmp=(ell*(ell+1))/OutputDenominator*const_fourpi
        if (State%CP%want_cl_2D_array) then
            fac=1
            fac(2) = sqrt(ctnorm)
            if (CTrans%NumSources > 2) then
                fac(3) = sqrt(ell*(ell+1)*CP%ALens) !Changed Dec18 for consistency
                do w_ix=3 + State%num_redshiftwindows+1,3 + State%num_redshiftwindows + CP%CustomSources%num_custom_sources
                    nscal= CP%CustomSources%custom_source_ell_scales(w_ix - State%num_redshiftwindows -3)
                    do i=1, nscal
                        fac(w_ix) = fac(w_ix)*(ell+i)*(ell-i+1)
                    end do
                    fac(w_ix) = sqrt(fac(w_ix))
                end do
            end if

            do w_ix=1, CTrans%NumSources - State%num_extra_redshiftwindows
                do w_ix2=w_ix,CTrans%NumSources - State%num_extra_redshiftwindows
                    iCl_Array(j,w_ix,w_ix2) =iCl_Array(j,w_ix,w_ix2) &
                        *fac(w_ix)*fac(w_ix2)*dbletmp
                    iCl_Array(j,w_ix2,w_ix) = iCl_Array(j,w_ix,w_ix2)
                end do
            end do
        end if

        iCl_scalar(j,C_Temp)  =  iCl_scalar(j,C_Temp)*dbletmp
        iCl_scalar(j,C_E) =  iCl_scalar(j,C_E)*dbletmp*ctnorm
        iCl_scalar(j,C_Cross) =  iCl_scalar(j,C_Cross)*dbletmp*sqrt(ctnorm)
        if (CTrans%NumSources>2) then
            iCl_scalar(j,C_Phi) = CP%ALens*iCl_scalar(j,C_Phi)*const_fourpi*ell**4
            !The lensing power spectrum computed is l^4 C_l^{\phi\phi}
            !We put pix extra factors of l here to improve interpolation in l
            iCl_scalar(j,C_PhiTemp) = sqrt(CP%ALens)*  iCl_scalar(j,C_PhiTemp)*const_fourpi*ell**3
            !Cross-correlation is CTrans%ls%l^3 C_l^{\phi T}
            iCl_scalar(j,C_PhiE) = sqrt(CP%ALens)*  iCl_scalar(j,C_PhiE)*const_fourpi*ell**3*sqrt(ctnorm)
            !Cross-correlation is CTrans%ls%l^3 C_l^{\phi E}
        end if
    end do
#ifndef __INTEL_COMPILER
    !$OMP END PARALLEL DO
#endif

    end subroutine CalcScalCls
```
我们需要的是TT,TE与EE,camb并没有提供low-ell-EB与BB部分的计算

这里只计算CMB

使用linear model  

```fortran
subroutine TimeSourcesToCl(ThisCT)
Type(ClTransferData) :: ThisCT 
integer q_ix
Type(TTimer) :: Timer

if (CP%WantScalars) ThisSources => State%ScalarTimeSources

if (DebugMsgs .and. Feedbacklevel > 0) call Timer%Start()

if (CP%WantScalars .and. WantLateTime &
    .and. (CP%NonLinear==NonLinear_Lens .or. CP%NonLinear==NonLinear_both) .and. global_error_flag==0) then
    call MakeNonlinearSources
    if (DebugMsgs .and. Feedbacklevel > 0) call Timer%WriteTime('Timing for NonLinear sources')
else
    ScaledSrc => ThisSources%LinearSrc
end if

if (global_error_flag==0) then
    call InitSourceInterpolation

    ExactClosedSum = State%curv > 5e-9_dl .or. State%scale < 0.93_dl

    max_bessels_l_index = ThisCT%ls%nl
    max_bessels_etak  = maximum_qeta

    if (CP%WantScalars) call GetLimberTransfers(ThisCT)
    ThisCT%max_index_nonlimber = max_bessels_l_index

    if (State%flat) call InitSpherBessels(ThisCT%ls, CP, max_bessels_l_index,max_bessels_etak )
    !This is only slow if not called before with same (or higher) Max_l, Max_eta_k
    !Preferably stick to Max_l being a multiple of 50

    call SetkValuesForInt(ThisCT)

    if (DebugMsgs .and. Feedbacklevel > 0) call WriteFormat('Set %d integration k values',ThisCT%q%npoints)

    !Begin k-loop and integrate Sources*Bessels over time
    !$OMP PARALLEL DO DEFAULT(SHARED), SCHEDULE(STATIC,4)
    do q_ix=1,ThisCT%q%npoints
        call SourceToTransfers(ThisCT, q_ix)
    end do !q loop
    !$OMP END PARALLEL DO

    if (DebugMsgs .and. Feedbacklevel > 0) call Timer%WriteTime('Timing for Integration')
end if

if (allocated(ddScaledSrc)) deallocate(ddScaledSrc)
if (associated(ScaledSrc) .and. .not. associated(ScaledSrc,ThisSources%LinearSrc)) then
    deallocate(ScaledSrc)
    nullify(ScaledSrc)
end if

!Final calculations for CMB output unless want the Cl transfer functions only.
if (.not. State%OnlyTransfer .and. global_error_flag==0) &
    call ClTransferToCl(State)

if (DebugMsgs .and. Feedbacklevel > 0) call Timer%WriteTime('Timing for final CL output')

end subroutine TimeSourcesToCl
```

IDM模型为flat

```fortran
subroutine DoFlatIntegration(IV, ThisCT, llmax)
implicit none
type(IntegrationVars) IV
Type(ClTransferData) :: ThisCT 
integer llmax
integer j
logical DoInt
real(dl) xlim,xlmax1
real(dl) tmin, tmax
real(dl) a2, J_l, aa(IV%SourceSteps), fac(IV%SourceSteps)
real(dl) xf, sums(ThisSources%SourceNum)
real(dl) qmax_int
integer bes_ix,n, bes_index(IV%SourceSteps)
integer custom_source_off, s_ix
integer nwin
real(dl) :: BessIntBoost
BessIntBoost = CP%Accuracy%AccuracyBoost*CP%Accuracy%BessIntBoost
custom_source_off = State%num_redshiftwindows + State%num_extra_redshiftwindows + 4
!     Find the position in the xx table for the x correponding to each
!     timestep
do j=1,IV%SourceSteps !Precompute arrays for this k
    xf=abs(IV%q*(State%tau0-State%TimeSteps%points(j)))
    bes_index(j)=BessRanges%IndexOf(xf)
    !Precomputed values for the interpolation
    bes_ix= bes_index(j)
    fac(j)=BessRanges%points(bes_ix+1)-BessRanges%points(bes_ix)
    aa(j)=(BessRanges%points(bes_ix+1)-xf)/fac(j)
    fac(j)=fac(j)**2*aa(j)/6
end do
do j=1,max_bessels_l_index
    if (ThisCT%ls%l(j) > llmax) return
    xlim=xlimfrac*ThisCT%ls%l(j)
    xlim=max(xlim,xlimmin)
    xlim=ThisCT%ls%l(j)-xlim
    if (full_bessel_integration .or. do_bispectrum) then
        tmin = State%TimeSteps%points(2)
    else
        xlmax1=80*ThisCT%ls%l(j)*BessIntBoost
        if (State%num_redshiftwindows>0 .and. CP%WantScalars) then
            xlmax1=80*ThisCT%ls%l(j)*8*BessIntBoost !Have to be careful if sharp spikes due to late time sources
        end if
        tmin=State%tau0-xlmax1/IV%q
        tmin=max(State%TimeSteps%points(2),tmin)
    end if
    tmax=State%tau0-xlim/IV%q
    tmax=min(State%tau0,tmax)
    tmin=max(State%TimeSteps%points(2),tmin)
    if (.not. CP%Want_CMB .and. .not. CP%Want_CMB_lensing) &
        tmin = max(tmin, State%ThermoData%tau_start_redshiftwindows)
    if (tmax < State%TimeSteps%points(2)) exit
    sums = 0
    !As long as we sample the source well enough, it is sufficient to
    !interpolate the Bessel functions only
    if (ThisSources%SourceNum==2) then
        !This is the innermost loop, so we separate the no lensing scalar case to optimize it
        do n= State%TimeSteps%IndexOf(tmin),min(IV%SourceSteps,State%TimeSteps%IndexOf(tmax))
            a2=aa(n)
            bes_ix=bes_index(n)
            J_l=a2*ajl(bes_ix,j)+(1-a2)*(ajl(bes_ix+1,j) - ((a2+1) &
                *ajlpr(bes_ix,j)+(2-a2)*ajlpr(bes_ix+1,j))* fac(n)) !cubic spline
            J_l = J_l*State%TimeSteps%dpoints(n)
            sums(1) = sums(1) + IV%Source_q(n,1)*J_l
            sums(2) = sums(2) + IV%Source_q(n,2)*J_l
        end do
    else
        qmax_int= max(850,ThisCT%ls%l(j))*3*BessIntBoost/State%tau0*1.2
        DoInt = .not. CP%WantScalars .or. IV%q < qmax_int
        !Do integral if any useful contribution to the CMB, or large scale effects
        if (DoInt) then
            if (CP%CustomSources%num_custom_sources==0 .and. State%num_redshiftwindows==0) then
                do n= State%TimeSteps%IndexOf(tmin),min(IV%SourceSteps,State%TimeSteps%IndexOf(tmax))
                    !Full Bessel integration
                    a2=aa(n)
                    bes_ix=bes_index(n)
                    J_l=a2*ajl(bes_ix,j)+(1-a2)*(ajl(bes_ix+1,j) - ((a2+1) &
                        *ajlpr(bes_ix,j)+(2-a2)*ajlpr(bes_ix+1,j))* fac(n)) !cubic spline
                    J_l = J_l*State%TimeSteps%dpoints(n)
                    !The unwrapped form is faster
                    sums(1) = sums(1) + IV%Source_q(n,1)*J_l
                    sums(2) = sums(2) + IV%Source_q(n,2)*J_l
                    sums(3) = sums(3) + IV%Source_q(n,3)*J_l
                end do
            else
                if (State%num_redshiftwindows>0) then
                    nwin = State%TimeSteps%IndexOf(State%ThermoData%tau_start_redshiftwindows)
                else
                    nwin = State%TimeSteps%npoints+1
                end if
                if (CP%CustomSources%num_custom_sources==0) then
                    do n= State%TimeSteps%IndexOf(tmin),min(IV%SourceSteps,State%TimeSteps%IndexOf(tmax))
                        !Full Bessel integration
                        a2=aa(n)
                        bes_ix=bes_index(n)
                        J_l=a2*ajl(bes_ix,j)+(1-a2)*(ajl(bes_ix+1,j) - ((a2+1) &
                            *ajlpr(bes_ix,j)+(2-a2)*ajlpr(bes_ix+1,j))* fac(n)) !cubic spline
                        J_l = J_l*State%TimeSteps%dpoints(n)
                        !The unwrapped form is faster
                        sums(1) = sums(1) + IV%Source_q(n,1)*J_l
                        sums(2) = sums(2) + IV%Source_q(n,2)*J_l
                        sums(3) = sums(3) + IV%Source_q(n,3)*J_l
                        if (n >= nwin) then
                            do s_ix = 4, ThisSources%SourceNum
                                sums(s_ix) = sums(s_ix) + IV%Source_q(n,s_ix)*J_l
                            end do
                        end if
                    end do
                else
                    do n= State%TimeSteps%IndexOf(tmin),min(IV%SourceSteps,State%TimeSteps%IndexOf(tmax))
                        !Full Bessel integration
                        a2=aa(n)
                        bes_ix=bes_index(n)
                        J_l=a2*ajl(bes_ix,j)+(1-a2)*(ajl(bes_ix+1,j) - ((a2+1) &
                            *ajlpr(bes_ix,j)+(2-a2)*ajlpr(bes_ix+1,j))* fac(n)) !cubic spline
                        J_l = J_l*State%TimeSteps%dpoints(n)
                        !The unwrapped form is faster
                        sums(1) = sums(1) + IV%Source_q(n,1)*J_l
                        sums(2) = sums(2) + IV%Source_q(n,2)*J_l
                        sums(3) = sums(3) + IV%Source_q(n,3)*J_l
                        sums(custom_source_off) = sums(custom_source_off) +  IV%Source_q(n,custom_source_off)*J_l
                        if (n >= nwin) then
                            do s_ix = 4, ThisSources%NonCustomSourceNum
                                sums(s_ix) = sums(s_ix) + IV%Source_q(n,s_ix)*J_l
                            end do
                        end if
                        do s_ix = custom_source_off+1, custom_source_off+CP%CustomSources%num_custom_sources -1
                            sums(s_ix) = sums(s_ix)  + IV%Source_q(n,s_ix)*J_l
                        end do
                    end do
                end if
            end if
        end if
        if (.not. DoInt .or. UseLimber(ThisCT%ls%l(j)) .and. CP%WantScalars) then
            !Limber approximation for small scale lensing (better than poor version of above integral)
            xf = State%tau0-(ThisCT%ls%l(j)+0.5_dl)/IV%q
            if (xf < State%TimeSteps%Highest .and. xf > State%TimeSteps%Lowest) then
                n=State%TimeSteps%IndexOf(xf)
                xf= (xf-State%TimeSteps%points(n))/(State%TimeSteps%points(n+1)-State%TimeSteps%points(n))
                sums(3) = (IV%Source_q(n,3)*(1-xf) + xf*IV%Source_q(n+1,3))*&
                    sqrt(const_pi/2/(ThisCT%ls%l(j)+0.5_dl))/IV%q
            else
                sums(3)=0
            end if
        end if
        if (.not. DoInt .and. ThisSources%NonCustomSourceNum>3) then
            if (any(ThisCT%limber_l_min(4:ThisSources%NonCustomSourceNum)==0 .or. &
                ThisCT%limber_l_min(4:ThisSources%NonCustomSourceNum) > j)) then
                !When CMB does not need integral but other sources do
                do n= State%TimeSteps%IndexOf(State%ThermoData%tau_start_redshiftwindows), &
                    min(IV%SourceSteps, State%TimeSteps%IndexOf(tmax))
                    !Full Bessel integration
                    a2 = aa(n)
                    bes_ix = bes_index(n)
                    J_l = a2 * ajl(bes_ix, j) + (1 - a2) * (ajl(bes_ix + 1, j) -&
                        ((a2 + 1) * ajlpr(bes_ix, j) + (2 - a2) * &
                        ajlpr(bes_ix + 1, j)) * fac(n)) !cubic spline
                    J_l = J_l * State%TimeSteps%dpoints(n)
                    sums(4) = sums(4) + IV%Source_q(n, 4) * J_l
                    do s_ix = 5, ThisSources%NonCustomSourceNum
                        sums(s_ix) = sums(s_ix) + IV%Source_q(n, s_ix) * J_l
                    end do
                end do
            end if
        end if
    end if
    ThisCT%Delta_p_l_k(:,j,IV%q_ix) = ThisCT%Delta_p_l_k(:,j,IV%q_ix) + sums
end do
end subroutine DoFlatIntegration
```

### 接下来我们将对这部分代码做python实现

In [16]:
# 导入包
import numpy as np
import matplotlib.pyplot as plt
import scipy

from astropy.constants import c
const_c = c.to('km/s').value

In [17]:
# 把z重构为向量函数 z = [dz0,dz1,dz2]
def function(t, z, kC1, O10, H0):
    # z[0] = z(t), z[1] = z'(t), z[2] = z''(t)
    dz1 = z[1]
    # 减少括号的使用,分为分子与分母
    up = H0**4 * kC1 * O10**2 * (z[0]**4+1) + 3 * H0**4 * O10**2 * z[0]**2 * (2 * kC1-3 * z[1]) \
        + H0**4 * O10**2 * z[0]**3 * (4 * kC1 - 3 * z[1]) - 3 * H0**4 * O10**2 * z[1] + 5 * H0**2 * O10 * z[1]**3\
            - kC1 * z[1]**4 + H0**2 * O10 * z[0] * (4 * H0**2 * kC1 * O10 - 9 * H0**2 * O10 * z[1] + 5 * z[1]**3)
    down = 2 * H0**2 * O10 * (1 + z[0])**2 * z[1]
    dz2 = up / down
    return [dz1, dz2]

In [18]:
# 求解原方程对应的z(t)与z'(t)
def solution(log_kC1, O20, H0):
    kC1 = 10**log_kC1
    O10 = 1 - O20
    t0 = 1 / H0
    # 求解区间
    tspan = (t0, 0)
    tn = np.linspace(t0, 0, 100000)
    # 从t0开始
    zt0 = [0, -H0]

    # t0给定初值
    z = scipy.integrate.solve_ivp(function, t_span=tspan, y0=zt0, t_eval=tn, method='RK45', args=(kC1, O10, H0))
    # z.y[0,:] = z(t), z.y[1,:] = z'(t)
    return z

In [19]:
class camb:
    def __init__(self, O20, log_kC1, H0):
        self.O20 = O20
        self.log_kC1 = log_kC1
        self.H0 = H0
        self.t0 = 1 / H0
        # dtau = dt / a
        self.tau0 = 1 / H0 * const_c # Mpc
        self.t_list = np.array(solution(log_kC1, O20, H0).t)
        self.z_list = np.array(solution(log_kC1, O20, H0).y[0, :])
    # (flat) initial power spectrum
    # from planck 2018
    def Pk(self ,k):
        As = 2.092e-9
        ns = 0.9647
        n_run = 0.0011
        n_runrun = 0.009
        k0 = 0.05

        lnrat = np.log(k / k0)
        return As * np.exp(lnrat * (ns - 1 + lnrat * (n_run / 2 + lnrat * n_runrun / 6)))
    # tau0 - tau
    def delta_tau(self, t):
        idx = np.searchsorted(self.t_list, t)
        int_value = -np.trapz(self.z_list[:idx], self.t_list[:idx])
        r = self.t0 - t + int_value
        return r * const_c # Mpc
    
    def Source_q(self, k):
        Source_q = np.zeros((2))
        # calculate Source_q
        k2 = k**2

        opacity = 1
        dopacity = 0
        visibility =
        dvisibility =
        ddvisibility =
        exptau =
        
        E2 = pig/4 + pigdot*(1/opacity)*(-5/8)
        Edot2 = (pigdot/4)*(1+(5/2)*(dopacity/opacity**2))
        Edot3 = 0
        
        phidot = (1 / 2) * (adotoa * (-dgpi - 2 * k2 * phi) + dgq * k - diff_rhopi + k * sigma * (gpres + grho)) / k2]
        sigmadot = -adotoa*sigma - 1/2*dgpi/k + k*phi
        polter = pig/10+9/15*E2
        polterdot = (1/10)*pigdot + (3/5)*Edot2
        polterddot = -2/25*adotoa*dgq/k - 4/75*adotoa* k*sigma - 4/75*dgpi - 2/75*dgrho/1 - 3/ 50*k*octgdot*1 + (1/25)*k*qgdot - 1/5 \
                        *k*Edot3 + (-1/10*pig + (7/10)* polter - 3/5*E2)*dopacity + (-1/10*pigdot + (7/10)*polterdot - 3/5*Edot2)*opacity

        ISW = 2 * phidot * exptau
        doppler = ((sigma + vb)*dvisibility + (sigmadot + vbdot)*visibility)/k
        monopole_source =  (-etak/k + 2*phi + clxg/4)*visibility
        quadrupole_source = (5 / 8)*(3*polter*ddvisibility + 6*polterdot*dvisibility + (k**2*polter + 3*polterddot)*visibility)/k**2
        Source_q[0] = ISW + doppler + monopole_source + quadrupole_source

        ang_dist = self.tau0
        Source_q[1] = visibility * polter * (15 / 8)/ (ang_dist ** 2 * k2)
        return Source_q
    
    def Delta_p_l_k(self, p, l, k):
        J_l_list = np.array([])
        # tau from tmin to tmax
        for t in self.t_list:
            delta_tau = self.delta_tau(t)
            xf = k * (delta_tau)
            J_l = scipy.special.spherical_jn(l, xf)
            J_l_list = np.append(J_l_list, J_l)
        int_value = -np.trapz(self.Source_q(k)[p-1] * J_l_list, self.t_list)
        return int_value
    # integrate dk/k * Delta_l_q ** 2 * Pk 
    def iCl_scalar(self, l, method):
        k_max = np.inf
        if method == 'TT':
            def integrand(k, l):
                return (self.Delta_p_l_k(1, l, k) ** 2 * self.Pk(k) / k)
            return scipy.integrate.quad(integrand, 0, k_max, args=(l))[0]
        elif method == 'EE':
            def integrand(k, l):
                return (self.Delta_p_l_k(2, l, k) ** 2 * self.Pk(k) / k)
            return scipy.integrate.quad(integrand, 0, k_max, args=(l))[0]
        elif method == 'TE':
            def integrand(k, l):
                return (self.Delta_p_l_k(1, l, k) * self.Delta_p_l_k(2, l, k) * self.Pk(k) / k)
            return scipy.integrate.quad(integrand, 0, k_max, args=(l))[0]
        else:
            print('method not supported')
            return 0
    # output l(l+1)Cl/2pi
    def Cl_scalar(self, l, method):
        ctnorm = (l * l - 1) * (l + 2) * l
        dbletmp = (l * (l + 1)) * 2
        if method == 'TT':
            return self.iCl_scalar(l, method) * dbletmp
        elif method == 'EE':
            return self.iCl_scalar(l, method) * ctnorm * dbletmp
        elif method == 'TE':
            return self.iCl_scalar(l, method) * np.sqrt(ctnorm) * dbletmp
        else:
            print('method not supported')
            return 0

In [20]:
result = camb(0.28, -5, 70)
result.Cl_scalar(100, 'TT')